In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

### Specify Features and Data Types

In [2]:
column_types = {'md5_cust_party_key': 'object', 'province_cd': 'object', 'z_age': 'float64', 'Gender_CD': 'object', 
                'z_census_household_1p_pct': 'float64', 'z_census_education_high_pct': 'float64', 
                'z_census_purchase_household': 'float64', 'z_census_purchase_capita': 'float64', 
                'z_census_household_cnt': 'float64', 'prod_monodual_cd': 'object', 'multiplay_cnt': 'float64', 
                'z_line_cnt': 'float64', 'z_sim_cnt': 'float64', 'fixed_prod_cat1_ind': 'float64', 'tenure_fixed_month': 'float64', 
                'tenure_mobile_month': 'float64', 'z_line_voice_cat1_cnt': 'float64', 'fixed_data_cat1_ind': 'float64', 
                'fixed_data_cat2_ind': 'float64', 'z_fixed_prod_cat2_cnt': 'float64', 'z_fixed_prod_cat1_cnt': 'float64', 
                'z_fixed_data_cat3_cnt': 'float64', 'fixed_prod_cat3_cnt': 'float64', 'device_smartphone_cnt': 'float64', 
                'z_mobile_voice_cat1_cnt': 'float64', 'z_mobile_data_cat1_cnt': 'float64', 'mobile_data_cat2_cnt': 'float64', 
                'z_mobile_voice_cat3_cnt': 'float64', 'z_mobile_data_cat3_cnt': 'float64', 'z_usg_fv_3m_avg': 'float64', 
                'z_usg_fd_mb_1m_sum': 'float64', 'z_usg_fd_mb_3m_avg': 'float64', 'z_usg_mv_ib_a_3m_avg': 'float64', 
                'z_usg_md_sms_ib_a_3m_avg': 'float64', 'z_usg_md_ib_mb_3m_avg': 'float64', 'payment_method_cash_cnt': 'float64', 
                'customer_value_cd': 'object', 'z_rev_1m_sum': 'float64', 'z_device_netcube_cnt': 'float64', 
                'z_tariff_netcube_cnt': 'float64', 'z_min_Prog_Max_BB_Down': 'float64', 'z_line_Fib2h_CNT': 'float64', 
                'z_min_Speed_Product_KBit': 'float64', 'z_Max_Speed_Missing_KBit': 'float64', 
                'z_Min_Speed_Reserve_KBit': 'float64', 'z_Max_DSL_OOS_PCT': 'float64', 'z_PR_Relocation_CNT': 'float64', 
                'z_PR_Relocation_Days': 'float64', 'z_PR_ActivationSupportOpt_CNT': 'float64', 
                'z_PR_ActivationSupportOpt_Days': 'float64', 'z_PR_DeactivationThreat_CNT': 'float64', 
                'z_PR_DeactivationSupport_CNT': 'float64', 'z_PR_DeactivationProdOpt_CNT': 'float64', 
                'z_PR_DeactivationProdOpt_Days': 'float64', 'z_PR_OtherWOTopic_CNT': 'float64', 
                'z_PR_OtherWOTopic_Days': 'float64', 'z_PR_AddressChange_CNT': 'float64', 
                'z_PR_AddressChange_Days': 'float64', 'z_PR_ServiceDisruption_CNT': 'float64', 
                'z_PR_ServiceDisruption_Days': 'float64', 'z_PR_BasketSupport_CNT': 'float64', 
                'z_PR_BasketSupport_Days': 'float64', 'z_PR_SellingSalesSupport_CNT': 'float64', 
                'z_PR_SellingSalesSupport_Days': 'float64', 'z_PR_DigitalUsage_CNT': 'float64', 
                'z_PR_DigitalUsage_Days': 'float64', 'z_TNPS_Last_Days': 'float64', 'z_TNPS_Score_Avg': 'float64'}

### Parse Dates, Specify Special NA Format, and Load Test Data

In [3]:
parser = lambda date: datetime.strptime(date, '%Y%m')

In [4]:
test_data = pd.read_csv('input.csv', dtype=column_types, na_values= ['*******'], parse_dates=['report_period_m_cd'], date_parser=parser)

In [5]:
churned = pd.read_json('ground_truth.json', typ='series')

In [6]:
def label_class (row):
    if row['md5_cust_party_key'] in churned['churned']:
        return 1
    elif row['md5_cust_party_key'] in churned['nonChurned']:
        return 0

In [7]:
test_data['target_ind'] = test_data.apply(lambda row: label_class(row), axis=1)

In [8]:
test_data = test_data.fillna(test_data.mean())

C:\Users\ahmed\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  """Entry point for launching an IPython kernel.


### Data Cleaning: Delete All Rows with Any Missing Data

In [9]:
no_nan = test_data

In [9]:
n_rows = len(test_data.index)
n_rows_no_na = len(no_nan.index)
print('Number of Rows with NAs Dropped: ' + str(n_rows_no_na))

Number of Rows with NAs Dropped: 420135


In [10]:
print('Number of Rows (Original): ' + str(n_rows))
print('Number of Rows (Cleaned): ' + str(n_rows_no_na))
print('Percentage of Retained Data After Cleaning: ' + str(n_rows_no_na/n_rows))
print('Percentage of Class 1 (Original): ' + str(test_data['target_ind'].sum()/n_rows))
print('Percentage of Class 1 (Cleaned): ' + str(no_nan['target_ind'].sum()/n_rows_no_na))
print('Test Data Shape (Original): ', test_data.shape)
print('Test Data Shape (Cleaned): ', no_nan.shape)

Number of Rows (Original): 454404
Number of Rows (Cleaned): 420135
Percentage of Retained Data After Cleaning: 0.9245847307682151
Percentage of Class 1 (Original): 0.020466369134074524
Percentage of Class 1 (Cleaned): 0.019998333868875482
Test Data Shape (Original):  (454404, 70)
Test Data Shape (Cleaned):  (420135, 70)


### Drop Uninformative Features

In [10]:
no_nan = no_nan.drop(['report_period_m_cd','md5_cust_party_key'], axis=1)

In [11]:
pd.set_option('display.max_columns', 500)

### One-Hot Encoding of Categorical Features

In [12]:
one_hot_columns = {'province_cd': 'object', 'Gender_CD': 'object', 'prod_monodual_cd': 'object'}

In [13]:
no_nan = pd.get_dummies(no_nan, columns=one_hot_columns.keys(), dtype=float)

#### Drop Gender_CD_M (Complement of Gender_CD_F) and prod_monodual_cd_F (Complement of prod_monodual_cd_D)

In [14]:
no_nan = no_nan.drop(['Gender_CD_M', 'prod_monodual_cd_F'], axis=1)

### Label Encoding of customer_value_cd Column

In [15]:
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=["NEW", "LOW", "MEDIUM", "HIGH", "HIGHEST"], ordered=True)
no_nan['customer_value_cd'] = no_nan['customer_value_cd'].astype(cat_type)
no_nan['customer_value_cd_cat'] = no_nan['customer_value_cd'].cat.codes
no_nan = no_nan.drop(['customer_value_cd'], axis=1)

In [16]:
print('Final Number of Features + Class: ')
no_nan.shape

Final Number of Features + Class: 


(454404, 76)

In [17]:
no_nan = no_nan.fillna(no_nan.mean())

### Save

In [18]:
X = no_nan.drop('target_ind', axis=1)
y = no_nan['target_ind']

In [19]:
X.to_csv('X_test_comp.csv', index=False)
y.to_csv('y_test_comp.csv', index=False)